In [1]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://graphs-neo4j:7687",
  auth=basic_auth("neo4j", "password"))

## Consultas básicas

La consulta de lectura básica de Cypher se compone de las cláusulas:

- `MATCH`: Especifica el patrón a recuperar en el grafo, es la cláusula análoga a `FROM` en SQL
- `WHERE`: Especifica los filtros a aplicar sobre los resultados obtenidos en `MATCH`
- `RETURN`: Especifica la estructura de los datos retornados, es la cláusula análoga a `SELECT` en SQL


In [ ]:
MATCH (n:User) WHERE n.screen_name = 'NASA' RETURN n; 

In [ ]:
MATCH (n:User) - [r] - (m) WHERE n.screen_name = 'NASA' RETURN m;

In [ ]:
MATCH (n:User) - [r:FOLLOWS] - (m) WHERE n.screen_name = 'NASA' RETURN m;

In [ ]:
MATCH (n:User) <- [r:FOLLOWS] - (m:User) WHERE n.screen_name = 'neo4j' RETURN m;

En Cypher se pueden combinar consultas utilizando variables definidas en una cláusula `MATCH` en otra clásula
`MATCH` posterior.

In [ ]:
MATCH (y1) -- (x:User {screen_name: 'NASA'}) -- (y2) MATCH (y1) -[r]->(y2) RETURN r;

## CRUD

Para crear un vértice se declara una variable para el vértice y se definen sus propiedades en forma de diccionario utilizando la cláusula `CREATE`

In [ ]:
CREATE (n:User {name: 'test', screen_name: 'test'})

Cuando se utiliza `CREATE` el sistema no busca elementos ya existentes en el grafo sino que intenta crear todo el patrón desde cero.

In [ ]:
CREATE (n:User {name: 'test', screen_name: 'test'}) - [r:POSTS] -> (t:Tweet {text: 'hello there'})

Para que el sistema busque elementos existentes en el patrón se debe utilizar la clásula `MATCH` junto con la cláusula `CREATE`

In [ ]:
MATCH (n:User {name: 'test', screen_name: 'test'}) CREATE (n) - [r:POSTS] -> (t:Tweet {text: 'hello there'})

Para actualizar elementos se utiliza la cláusula `WHERE` aplicada a los elementos definidos en la consulta `MATCH`

In [ ]:
MATCH (n:User {name: 'test', screen_name: 'test'}) SET n.name = 'test modified'

Para eliminar se utiliza la cláusula `DELETE`, nótese que debido a la integridad referencial por defecto no se eliminan vértices que tienen aristas y las aristas deben ser eliminadas manualmente por el usuario. Para eliminar las aristas en cascada se puede utilizar la
cláusula `DETACH`

In [ ]:
MATCH (n:User {screen_name: 'test'}) [DETACH] DELETE n;

## Consultas avanzadas

Cypher provee varias de las cláusulas existentes en SQL como `ORDER BY`, `LIMIT`, `SKIP`, etc. Además existent las funciones de agregación comunes como `COUNT`, `SUM`, `AVG`, etc. 

In [ ]:
MATCH (u:User {screen_name: 'neo4j'})<-[r:MENTIONS]-(t:Tweet)-[r2:TAGS]->(h:Hashtag)
RETURN h.name as hashtag, COUNT(t)
ORDER BY COUNT(t) DESC

Cypher permite buscar caminos dentro del grafo utilizando el operador `*n` donde `n` es la longitud del camino buscado, este operador
se aplica sobre las aristas

In [ ]:
MATCH p=(n:Tweet {id: 1370785056361381889})<-[:REPLY_TO*]-(:Tweet)
WITH collect(RELATIONSHIPS(p)) AS rel_list
UNWIND rel_list AS list
UNWIND list AS link
WITH DISTINCT(link)
RETURN startNode(link), link, endNode(link)